In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os

In [2]:
# random seed --> reproducibility
import random
random.seed(42)
torch.manual_seed(42)

In [3]:
# transforms
data_transforms = {
    'train': transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(5),    
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]),

    'test': transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
}

In [4]:
train_dir = '../input/chest-xray-pneumonia/chest_xray/train'
test_dir = '../input/chest-xray-pneumonia/chest_xray/test'

In [5]:
train_data = datasets.ImageFolder(root=train_dir, transform=data_transforms['train'])
test_data = datasets.ImageFolder(root=test_dir, transform=data_transforms['test'])

In [6]:
train_data

Dataset ImageFolder
    Number of datapoints: 5216
    Root location: ../input/chest-xray-pneumonia/chest_xray/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               RandomRotation(degrees=[-5.0, 5.0], interpolation=nearest, expand=False, fill=0)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [7]:
# DataLoader object creation for batch 
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size = 64,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_data,
                                           batch_size = 64,
                                           shuffle=False)

In [8]:
# example
iterate = iter(train_loader)
sample, labels = next(iterate)

In [9]:
sample.shape

torch.Size([64, 3, 224, 224])

In [10]:
labels

tensor([1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1])

In [11]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16,self).__init__()
        
        self.features = torch.nn.Sequential(
            # conv layer 1
            nn.Conv2d(in_channels=3,
                      out_channels=6,
                      kernel_size=3,
                      padding=1),   
            nn.ReLU(inplace=True),
            
            # conv layer 2
            nn.Conv2d(in_channels=6,
                      out_channels=10,
                      kernel_size=3,
                      padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(inplace=True),
            
            # conv layer 3
            nn.Conv2d(in_channels=10,
                      out_channels=12,
                      kernel_size=3,
                      padding=1),   
            nn.ReLU(inplace=True),
            
            # conv layer 4
            nn.Conv2d(in_channels=12,
                      out_channels=14,
                      kernel_size=3,
                      padding=1), 
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(inplace=True),
            
            # conv layer 5
            nn.Conv2d(in_channels=14,
                      out_channels=16,
                      kernel_size=3,
                      padding=1),   
            nn.ReLU(inplace=True),
            
            # conv layer 6
            nn.Conv2d(in_channels=16,
                      out_channels=18,
                      kernel_size=3,
                      padding=1),   
            nn.ReLU(inplace=True),
            
            # conv layer 7
            nn.Conv2d(in_channels=18,
                      out_channels=20,
                      kernel_size=3,
                      padding=1), 
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(inplace=True),
            
            # conv layer 8
            nn.Conv2d(in_channels=20,
                      out_channels=22,
                      kernel_size=3,
                      padding=1),   
            nn.ReLU(inplace=True),
            
            # conv layer 9
            nn.Conv2d(in_channels=22,
                      out_channels=24,
                      kernel_size=3,
                      padding=1),   
            nn.ReLU(inplace=True),
            
            # conv layer 10
            nn.Conv2d(in_channels=24,
                      out_channels=26,
                      kernel_size=3,
                      padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(inplace=True),
            
            # conv layer 11
            nn.Conv2d(in_channels=26,
                      out_channels=28,
                      kernel_size=3,
                      padding=1),   
            nn.ReLU(inplace=True),
            
            # conv layer 12
            nn.Conv2d(in_channels=28,
                      out_channels=30,
                      kernel_size=3,
                      padding=1),   
            nn.ReLU(inplace=True),
            
            # conv layer 13
            nn.Conv2d(in_channels=30,
                      out_channels=32,
                      kernel_size=3,
                      padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(inplace=True))
                
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
            
        self.classifier = torch.nn.Sequential(
            #linear layer 1
            nn.Linear(32*7*7, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
        
            #linear layer 2
            nn.Linear(512,512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
        
            #linear layer 3
            nn.Linear(512,2))
        
    def forward(self,x):
        # convolution
        x = self.features(x)
        
        # adaptive average pooling
        x = self.avgpool(x)
        
        #flatten
        x = x.view(-1, 32*7*7)
        
        # fully connected layer
        x = self.classifier(x)
        
        return x

In [12]:
# model -- class object
model = VGG16()

In [13]:
model

VGG16(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(6, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): ReLU(inplace=True)
    (5): Conv2d(10, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(12, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): ReLU(inplace=True)
    (10): Conv2d(14, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(16, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(18, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): ReLU(inpl

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,115,490 trainable parameters


In [15]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
# total iteration
len(train_loader)

82

In [17]:
total_batches = len(train_loader)

for epoch in range(10):
    for i , (images, labels) in enumerate(train_loader):
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 40 == 0:
            print(f'Epoch [{epoch+1}/{10}], step [{i+1}/{total_batches}], loss:{loss.item():.4f}')
            
print('Finished Training')    

Epoch [1/10], step [40/82], loss:0.6662
Epoch [1/10], step [80/82], loss:0.6052
Epoch [2/10], step [40/82], loss:0.6268
Epoch [2/10], step [80/82], loss:0.5268
Epoch [3/10], step [40/82], loss:0.5108
Epoch [3/10], step [80/82], loss:0.5284
Epoch [4/10], step [40/82], loss:0.6001
Epoch [4/10], step [80/82], loss:0.5188
Epoch [5/10], step [40/82], loss:0.6200
Epoch [5/10], step [80/82], loss:0.6154
Epoch [6/10], step [40/82], loss:0.5239
Epoch [6/10], step [80/82], loss:0.4965
Epoch [7/10], step [40/82], loss:0.6531
Epoch [7/10], step [80/82], loss:0.7041
Epoch [8/10], step [40/82], loss:0.5538
Epoch [8/10], step [80/82], loss:0.5109
Epoch [9/10], step [40/82], loss:0.6369
Epoch [9/10], step [80/82], loss:0.6530
Epoch [10/10], step [40/82], loss:0.4899
Epoch [10/10], step [80/82], loss:0.6027
Finished Training


In [18]:
# save this model
torch.save(model.state_dict(), 'modelVGG16.pth')

In [19]:
# evaluation and testing 
with torch.no_grad():
    n_correct =0
    n_samples =0
    
    for images, labels in test_loader:
        # forward -- softmax prediction 
        outputs = model(images)
        
        # actual prediction
        # value, index
        _, predictions = torch.max(outputs, 1) # multiclass pred
        n_samples += labels.shape[0]
        n_correct += (predictions==labels).sum().item()
        
    acc = 100.0 * n_correct / n_samples
    print(acc)

62.5


### Initializing parameters to compare evaluation

In [20]:
## with initializing parameters
def initialize_parameters(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight.data, nonlinearity='relu')
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))
        nn.init.constant_(m.bias.data, 0)

In [21]:
model.apply(initialize_parameters)

VGG16(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(6, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): ReLU(inplace=True)
    (5): Conv2d(10, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(12, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): ReLU(inplace=True)
    (10): Conv2d(14, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(16, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(18, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): ReLU(inpl

In [22]:
total_batches = len(train_loader)

for epoch in range(10):
    for i , (images, labels) in enumerate(train_loader):
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 40 == 0:
            print(f'Epoch [{epoch+1}/{10}], step [{i+1}/{total_batches}], loss:{loss.item():.4f}')
            
print('Finished Training')    

Epoch [1/10], step [40/82], loss:0.2430
Epoch [1/10], step [80/82], loss:0.2869
Epoch [2/10], step [40/82], loss:0.0822
Epoch [2/10], step [80/82], loss:0.0880
Epoch [3/10], step [40/82], loss:0.1925
Epoch [3/10], step [80/82], loss:0.1317
Epoch [4/10], step [40/82], loss:0.1692
Epoch [4/10], step [80/82], loss:0.0870
Epoch [5/10], step [40/82], loss:0.0787
Epoch [5/10], step [80/82], loss:0.0958
Epoch [6/10], step [40/82], loss:0.0455
Epoch [6/10], step [80/82], loss:0.0420
Epoch [7/10], step [40/82], loss:0.0127
Epoch [7/10], step [80/82], loss:0.1324
Epoch [8/10], step [40/82], loss:0.0538
Epoch [8/10], step [80/82], loss:0.0568
Epoch [9/10], step [40/82], loss:0.0665
Epoch [9/10], step [80/82], loss:0.0600
Epoch [10/10], step [40/82], loss:0.0817
Epoch [10/10], step [80/82], loss:0.1431
Finished Training


In [23]:
# save this model
torch.save(model.state_dict(), 'modelVGG16_initialparam.pth')

In [24]:
# evaluation and testing 
with torch.no_grad():
    n_correct =0
    n_samples =0
    
    for images, labels in test_loader:
        # forward -- softmax prediction 
        outputs = model(images)
        
        # actual prediction
        # value, index
        _, predictions = torch.max(outputs, 1) # multiclass pred
        n_samples += labels.shape[0]
        n_correct += (predictions==labels).sum().item()
        
    acc = 100.0 * n_correct / n_samples
    print(acc)

78.68589743589743
